Note: you may need to restart the kernel to use updated packages.


Requirement already up-to-date: torch in /usr/local/anaconda3/lib/python3.8/site-packages (1.11.0)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%run tools/train.py

ImportError: cannot import name 'utils' from 'torchvision.models' (/usr/local/anaconda3/lib/python3.8/site-packages/torchvision/models/__init__.py)

In [14]:
import torchvision

In [18]:
pip install torchvision==0.7

     |████████████████████████████████| 5.9 MB 770 kB/s eta 0:00:01
     |████████████████████████████████| 748.8 MB 34 kB/s s eta 0:00:01    |█▉                              | 43.2 MB 21.4 MB/s eta 0:00:33     |████████████▊                   | 297.1 MB 31.4 MB/s eta 0:00:15     |██████████████▉                 | 347.7 MB 21.4 MB/s eta 0:00:19     |███████████████▊                | 367.4 MB 21.4 MB/s eta 0:00:18     |████████████████                | 372.3 MB 21.4 MB/s eta 0:00:18     |████████████████▏               | 379.4 MB 33.3 MB/s eta 0:00:12
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'RECORD'
Consider using t

In [20]:
%run tools/train.py

ModuleNotFoundError: No module named 'easydict'

In [21]:
pip install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=20680395b200c2d0d45131bad32897eae9ae024f49a6dcef656badbc6fb93e46
  Stored in directory: /home/scx/.cache/pip/wheels/d3/e0/e9/305e348717e399665119bd012510d51ff4f22d709ff60c3096
Successfully built easydict
Note: you may need to restart the kernel to use updated packages.


In [22]:
%run tools/train.py

usage: train.py [-h] [--weights WEIGHTS] [--resume RESUME] [--cfg CFG_FILE]
                [--set ...] [--method METHOD] [--exp_name EXP_NAME]

Train script.

optional arguments:
  -h, --help           show this help message and exit
  --weights WEIGHTS    initialize with specified model parameters
  --resume RESUME      initialize with saved solver status
  --cfg CFG_FILE       optional config file
  --set ...            set config keys
  --method METHOD      set the method to use
  --exp_name EXP_NAME  the experiment name


SystemExit: 1

In [23]:
%run tools/train.py

usage: train.py [-h] [--weights WEIGHTS] [--resume RESUME] [--cfg CFG_FILE]
                [--set ...] [--method METHOD] [--exp_name EXP_NAME]

Train script.

optional arguments:
  -h, --help           show this help message and exit
  --weights WEIGHTS    initialize with specified model parameters
  --resume RESUME      initialize with saved solver status
  --cfg CFG_FILE       optional config file
  --set ...            set config keys
  --method METHOD      set the method to use
  --exp_name EXP_NAME  the experiment name


SystemExit: 1

In [140]:
import torch
import argparse
import os
import numpy as np
from torch.backends import cudnn
from model import model
from config.config import cfg, cfg_from_file, cfg_from_list
from prepare_data import *
import sys
import pprint

def parse_args():
    """
    Parse input arguments
    """
    parser = argparse.ArgumentParser(description='Train script.')
    parser.add_argument('--weights', dest='weights',
                        help='initialize with specified model parameters',
                        default=None, type=str)
    parser.add_argument('--resume', dest='resume',
                        help='initialize with saved solver status',
                        default=None, type=str)
    parser.add_argument('--cfg', dest='cfg_file',
                        help='optional config file',
                        default="./experiments/config/Office-31/CAN/office31_train_amazon2dslr_cfg.yaml", type=str)
    parser.add_argument('--set', dest='set_cfgs',
                        help='set config keys', default=None,
                        nargs=argparse.REMAINDER)
    parser.add_argument('--method', dest='method',
                        help='set the method to use', 
                        default='CAN', type=str)
    parser.add_argument('--exp_name', dest='exp_name',
                        help='the experiment name', 
                        default='office31_a2d', type=str)


    if len(sys.argv) == 1:
        parser.print_help()
        sys.exit(1)
    args = parser.parse_args(args=[])
    return args

In [142]:
def train(args):
    bn_domain_map = {}

    # method-specific setting 
    if args.method == 'CAN': 
        from solver.can_solver import CANSolver as Solver
        dataloaders = prepare_data_CAN()
        num_domains_bn = 2

    elif args.method == 'MMD':
        from solver.mmd_solver import MMDSolver as Solver
        dataloaders = prepare_data_MMD()
        num_domains_bn = 2

    elif args.method == 'SingleDomainSource':
        from solver.single_domain_solver import SingleDomainSolver as Solver
        dataloaders = prepare_data_SingleDomainSource()
        num_domains_bn = 1

    elif args.method == 'SingleDomainTarget':
        from solver.single_domain_solver import SingleDomainSolver as Solver
        dataloaders = prepare_data_SingleDomainTarget()
        num_domains_bn = 1

    else:
        raise NotImplementedError("Currently don't support the specified method: %s." 
                                 % args.method)

    # initialize model
    model_state_dict = None
    fx_pretrained = True
    resume_dict = None

    if cfg.RESUME != '':
        resume_dict = torch.load(cfg.RESUME)
        model_state_dict = resume_dict['model_state_dict']
        fx_pretrained = False
    elif cfg.WEIGHTS != '':
        param_dict = torch.load(cfg.WEIGHTS)
        model_state_dict = param_dict['weights']
        bn_domain_map = param_dict['bn_domain_map']
        fx_pretrained = False

    net = model.danet(num_classes=cfg.DATASET.NUM_CLASSES, 
                 state_dict=model_state_dict,
                 feature_extractor=cfg.MODEL.FEATURE_EXTRACTOR, 
                 frozen=[cfg.TRAIN.STOP_GRAD], 
                 fx_pretrained=fx_pretrained, 
                 dropout_ratio=cfg.TRAIN.DROPOUT_RATIO,
                 fc_hidden_dims=cfg.MODEL.FC_HIDDEN_DIMS, 
                 num_domains_bn=num_domains_bn)

    net = torch.nn.DataParallel(net)
    if torch.cuda.is_available():
       net.cuda()

    # initialize solver
    train_solver = Solver(net, dataloaders, bn_domain_map=bn_domain_map, resume=resume_dict)

    # train 
    train_solver.solve()
    print('Finished!')



In [152]:
if __name__ == '__main__':
    cudnn.benchmark = True 
    args = parse_args()
    print('Called with args:')
    print(args.cfg_file)

    if args.cfg_file is not None:
        cfg_from_file(args.cfg_file)
    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs)

    if args.resume is not None:
        cfg.RESUME = args.resume 
    if args.weights is not None:
        cfg.MODEL = args.weights
    if args.exp_name is not None:
        cfg.EXP_NAME = args.exp_name 

    print('Using config:')
    pprint.pprint(cfg)

    cfg.SAVE_DIR = os.path.join(cfg.SAVE_DIR, cfg.EXP_NAME)
    print('Output will be saved to %s.' % cfg.SAVE_DIR)

    train(args)

Called with args:
./experiments/config/Office-31/CAN/office31_train_amazon2dslr_cfg.yaml
Using config:
{'ADAM': {'BETA1': 0.9, 'BETA2': 0.999},
 'CDD': {'ALIGNMENT_FEAT_KEYS': ['feat', 'probs'],
         'INTRA_ONLY': False,
         'KERNEL_MUL': [2, 2],
         'KERNEL_NUM': [5, 5],
         'LOSS_WEIGHT': 0.3},
 'CLUSTERING': {'BUDGET': 1000,
                'EPS': 0.005,
                'FEAT_KEY': 'feat',
                'FILTERING_THRESHOLD': 0.05,
                'HISTORY_LEN': 2,
                'SOURCE_BATCH_SIZE': 100,
                'TARGET_BATCH_SIZE': 100,
                'TARGET_DATASET_TYPE': 'SingleDatasetWithoutLabel'},
 'DATASET': {'DATAROOT': './experiments/dataset/Office-31',
             'NUM_CLASSES': 31,
             'SOURCE_NAME': 'amazon',
             'TARGET_NAME': 'dslr'},
 'DATA_TRANSFORM': {'FINESIZE': 224,
                    'FLIP': True,
                    'LOADSIZE': 256,
                    'NORMALIZE_MEAN': [0.485, 0.456, 0.406],
                 

FileNotFoundError: [Errno 2] No such file or directory: './experiments/ckpt/office31_a2d/ckpt_0_89.resume'